In [ ]:
%%bash
rm -rf *
git clone https://gitlab.com/mieiuminho/ds/aa2/tp.git
cd tp/data
bash ./download_data.sh
cd ../..
git clone https://github.com/pcko1/Deep-Drug-Coder.git
cd Deep-Drug-Coder
pip install gdown
pip install rdkit-pypi
pip install .
python -m pip install git+https://github.com/EBjerrum/molvecgen
cd ..
gdown 'https://drive.google.com/uc?id=18lawb6YxnMH4GPEZ-r2CeqcCqStWVRMA'
pip install tqdm
pip uninstall --yes pytorch
pip uninstall --yes torchvision
pip uninstall --yes torchaudio
pip install torch==1.8.0+cu111 torchvision==0.9.0+cu111 torchaudio==0.8.0 -f https://download.pytorch.org/whl/torch_stable.html

Processing /content/Deep-Drug-Coder
  Created wheel for ddc-pub: filename=ddc_pub-0.3-cp37-none-any.whl size=28958 sha256=647d5f3e3ceb5d81c96ae266dde8403fd8c8f0c751ec212202a18d970ae94000
  Stored in directory: /root/.cache/pip/wheels/de/62/f7/b56013272b86cd2568c39d10d1f73282e260a5bcc21df65b39
  Created wheel for gast: filename=gast-0.2.2-cp37-none-any.whl size=7557 sha256=be517c8281f222fbcc8427880f5cef2ce63366cb37bd44c9e22b82a0240e7714
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built ddc-pub gast
  Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
  Found existing installation: h5py 3.1.0
    Uninstalling h5py-3.1.0:
      Successfully uninstalled h5py-3.1.0
  Found existing installation: tensorboard 2.5.0
    Uninstalling tensorboard-2.5.0:
      Successfully uninstalled tensorboard-2.5.0
  Found existing installation: tensorflow-estimator 2.5

Cloning into 'tp'...
--2021-06-06 08:31:44--  https://media.githubusercontent.com/media/molecularsets/moses/master/data/train.csv
Resolving media.githubusercontent.com (media.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.111.133, ...
Connecting to media.githubusercontent.com (media.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 67916778 (65M) [text/plain]
Saving to: ‘train.csv’

     0K .......... .......... .......... .......... ..........  0% 4.19M 15s
    50K .......... .......... .......... .......... ..........  0% 4.89M 14s
   100K .......... .......... .......... .......... ..........  0% 17.2M 11s
   150K .......... .......... .......... .......... ..........  0% 21.5M 9s
   200K .......... .......... .......... .......... ..........  0% 7.95M 9s
   250K .......... .......... .......... .......... ..........  0% 32.5M 8s
   300K .......... .......... .......... .......... ..........  0% 23.6

In [ ]:
!mkdir output

In [ ]:
with open('latent_rep', 'r') as f:
  print(len(f.readline().split(',')))

FileNotFoundError: ignored

In [ ]:
from ddc_pub import ddc_v3 as ddc
from rdkit import Chem
import numpy as np
import os
import sys
import tensorflow as tf
import torch.nn as nn
import torch
from torch.utils import data
from torch.utils.data import DataLoader
from tqdm import tqdm
import torch.autograd as autograd
import time
import gc
import json

def load_model():
    model = ddc.DDC(model_name='moses_pretrained')
    return model


def encode(smiles_file, total_size=10000, batch_size=2048, output_latent_file_path='latent_rep'):
    """
    Converter SMILES para representação em vetor latente
    """
    model = load_model()

    # Input SMILES
    smiles_in = []
    curr_batch = []
    total_written = 0
    with open(output_latent_file_path, 'w') as f:
      with open(smiles_file, "r") as file:
          line = file.readline()
          line = file.readline()
          while line:
              smi = line.strip('\n').split(',')[0]

              curr_batch.append(smi)

              if len(curr_batch) % batch_size == 0:
                mols_in = [Chem.rdchem.Mol.ToBinary(Chem.MolFromSmiles(smiles)) for smiles in curr_batch]
                latent = model.transform(model.vectorize(mols_in))
                strlat = [[str(val) for val in lval[0]] for lval in latent]
                batch_content = [','.join(val) + '\n' for val in strlat]
                f.writelines(batch_content)
                total_written += len(curr_batch)
                curr_batch.clear()
                print('[total of %d out of %d]' % (total_written, total_size))
                if total_written > total_size:
                  break
              
              line = file.readline()

          if len(curr_batch) > 0:
            mols_in = [Chem.rdchem.Mol.ToBinary(Chem.MolFromSmiles(smiles)) for smiles in curr_batch]
            latent = model.transform(model.vectorize(mols_in))
            strlat = [[str(val) for val in lval[0]] for lval in latent]
            batch_content = [','.join(val) + '\n' for val in strlat]
            f.writelines(batch_content)
            curr_batch.clear()

    print('Encoding completed!')

def decode(latent_mols_file, input_size=10000,output_smiles_file_path=None, message=''):
    sys.stdout.flush()
    model = load_model()

    if output_smiles_file_path is None:
        output_smiles_file_path = os.path.join(os.path.dirname(latent_mols_file), 'decoded_smiles.smi')


    invalids = 0
    batch_size = 256    # decoding batch size
    n = input_size
    read_lines = []
    curr_size = 0
    with open(latent_mols_file, 'rU') as f:
      with open(output_smiles_file_path, 'w') as smiles_file:
          for indx in range(0, int(n) // int(batch_size)):
              read_lines.clear()
              curr_size = 1

              line = f.readline()
              while line:
                  print('curr size is ', curr_size)
                  read_lines.append( [float(val.strip('[] ')) for val in line.strip('\n').split(',') ])
                  line = f.readline()
                  curr_size = curr_size + 1

                  print('curr_size=', curr_size, ', batch_size=', batch_size)
                  if curr_size > batch_size:
                    line = None;

              if indx % 10 == 0:
                  print("Batch [%d/%d] decoded, [Invalids: %s]" % (indx, n // batch_size + 1, invalids))
                  sys.stdout.flush()
                  smiles_file.flush()

              smiles, _ = model.predict_batch(np.array(read_lines).reshape(batch_size*input_size,512), temp=0)
              for smi in smiles:

                  smiles_file.write(smi + '\n')
                  mol = Chem.MolFromSmiles(smi)
                  if not mol:
                      invalids += 1
    print("Decoding completed.")
    print("Total: [%d] Fraction Valid: [0.%d]" % (n, (n - invalids) / n * 100))

def read_latent_file(input_latent_file):
  with open(input_latent_file, 'r') as f:
    all_lines = np.array([ [float(val) for val in line.strip('\n').split(',')] for line in f.readlines()])
    all_lines.reshape(all_lines.shape[0], 512)
    return all_lines

class Discriminator(nn.Module):
    def __init__(self, data_shape):
        super(Discriminator, self).__init__()
        self.data_shape = data_shape

        self.model = nn.Sequential(
            nn.Linear(int(np.prod(self.data_shape)), 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 1),
        )

    def forward(self, mol):
        validity = self.model(mol)
        return validity

class LatentMolsDataset(data.Dataset):
    def __init__(self, latent_space_mols):
        self.data = latent_space_mols

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        return self.data[index]

class Generator(nn.Module):
    def __init__(self, data_shape, latent_dim=None):
        super(Generator, self).__init__()
        self.data_shape = data_shape

        # latent dim of the generator is one of the hyperparams.
        # by default it is set to the prod of data_shapes
        self.latent_dim = int(np.prod(self.data_shape)) if latent_dim is None else latent_dim

        def block(in_feat, out_feat, normalize=True):
            layers = [nn.Linear(in_feat, out_feat)]
            if normalize:
                layers.append(nn.BatchNorm1d(out_feat, 0.8))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            return layers

        self.model = nn.Sequential(
            *block(self.latent_dim, 128, normalize=False),
            *block(128, 256),
            *block(256, 512),
            *block(512, 1024),
            nn.Linear(1024, int(np.prod(self.data_shape))),
            # nn.Tanh() # expecting latent vectors to be not normalized
        )

    def forward(self, z):
        out = self.model(z)
        return out

class Sampler(object):
    def __init__(self, generator: Generator):
        self.set_generator(generator)

    def set_generator(self, generator):
        self.G = generator

    def sample(self, n):
        # Sample noise as generator input
        z = torch.cuda.FloatTensor(np.random.uniform(-1, 1, (n, self.G.latent_dim)))
        # Generate a batch of mols
        return self.G(z)

class TrainModelRunner:
    # Loss weight for gradient penalty
    

    def __init__(self, input_data_path, decode_mols_save_path='', n_epochs=2000, starting_epoch=1,
                 batch_size=64, lr=0.0002, b1=0.5, b2=0.999, lambda_gp = 10,  n_critic=5,
                 save_interval=1000, sample_after_training=30000):
        # init params
        self.input_data_path = input_data_path
        self.n_epochs = n_epochs
        self.starting_epoch = starting_epoch
        self.batch_size = batch_size
        self.lr = lr
        self.b1 = b1
        self.b2 = b2
        self.lambda_gp = lambda_gp
        self.n_critic = n_critic
        self.save_interval = save_interval
        self.sample_after_training = sample_after_training
        self.decode_mols_save_path = decode_mols_save_path

        # initialize dataloader
        latent_space_mols = read_latent_file(self.input_data_path)

        self.dataloader = DataLoader(LatentMolsDataset(latent_space_mols), shuffle=True,
                                                      batch_size=self.batch_size, drop_last=True)

        # load discriminator
        self.D = Discriminator((1,latent_space_mols.shape[1]))

        # load generator
        self.G = Generator((1,latent_space_mols.shape[1]))

        # initialize sampler
        self.Sampler = Sampler(self.G)

        # initialize optimizer
        self.optimizer_G = torch.optim.Adam(self.G.parameters(), lr=self.lr, betas=(self.b1, self.b2))
        self.optimizer_D = torch.optim.Adam(self.D.parameters(), lr=self.lr, betas=(self.b1, self.b2))

        # Tensor
        cuda = True if torch.cuda.is_available() else False
        if cuda:
            self.G.cuda()
            self.D.cuda()
        self.Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor

    def run(self):

        print("Training of GAN started.")
        sys.stdout.flush()

        batches_done = 0
        disc_loss_log = []
        g_loss_log = []

        for epoch in range(self.starting_epoch, self.n_epochs + self.starting_epoch):
            disc_loss_per_batch = []
            g_loss_log_per_batch = []
            for i, real_mols in enumerate(tqdm(self.dataloader)):

                # Configure input
                real_mols = real_mols.type(self.Tensor)
                

                # ---------------------
                #  Train Discriminator
                # ---------------------

                self.optimizer_D.zero_grad()

                # Generate a batch of mols from noise
                fake_mols = self.Sampler.sample(real_mols.shape[0])

                # Real mols
                real_validity = self.D(real_mols)
                # Fake mols
                fake_validity = self.D(fake_mols)
                # Gradient penalty
                gradient_penalty = self.compute_gradient_penalty(real_mols.data, fake_mols.data)
                # Adversarial loss
                d_loss = -torch.mean(real_validity) + torch.mean(fake_validity) + self.lambda_gp * gradient_penalty
                disc_loss_per_batch.append(d_loss.item())

                d_loss.backward()
                self.optimizer_D.step()
                self.optimizer_G.zero_grad()

                # Train the generator every n_critic steps
                if i % self.n_critic == 0:
                    # -----------------
                    #  Train Generator
                    # -----------------

                    # Generate a batch of mols
                    fake_mols = self.Sampler.sample(real_mols.shape[0])
                    # Loss measures generator's ability to fool the discriminator
                    # Train on fake images
                    fake_validity = self.D(fake_mols)
                    g_loss = -torch.mean(fake_validity)
                    g_loss_log_per_batch.append(g_loss.item())

                    g_loss.backward()
                    self.optimizer_G.step()

                    batches_done += self.n_critic

                # If last batch in the set
                if i == len(self.dataloader) - 1:
                    disc_loss_log.append([time.time(), epoch, np.mean(disc_loss_per_batch)])
                    g_loss_log.append([time.time(), epoch, np.mean(g_loss_log_per_batch)])

                    # Print and log
                    print(
                        "[Epoch %d/%d]  [Disc loss: %f] [Gen loss: %f] "
                        % (epoch, self.n_epochs + self.starting_epoch, disc_loss_log[-1][2], g_loss_log[-1][2])
                    )
                    sys.stdout.flush()

        # Sampling after training
        if self.sample_after_training > 0:
            print("Training finished. Generating sample of latent vectors")
            # sampling mode
            torch.no_grad()
            self.G.eval()

            latent = self.Sampler.sample(self.sample_after_training)
            latent = latent.detach().cpu().numpy().tolist()

            sampled_mols_save_path = os.path.join('output', 'sampled.json')
            with open(sampled_mols_save_path, 'w') as json_file:
                json.dump(latent, json_file)

            # decoding sampled mols
            print("Sampling done. Decoding latent vectors into SMILES")
            decode(sampled_mols_save_path, output_smiles_file_path=self.decode_mols_save_path)


    def compute_gradient_penalty(self, real_samples, fake_samples):
        """Calculates the gradient penalty loss for WGAN GP"""
        # Random weight term for interpolation between real and fake samples
        alpha = self.Tensor(np.random.random((real_samples.size(0), 1)))

        # Get random interpolation between real and fake samples
        interpolates = (alpha * real_samples + ((1 - alpha) * fake_samples)).requires_grad_(True)
        d_interpolates = self.D(interpolates)
        fake = self.Tensor(real_samples.shape[0], 1).fill_(1.0)

        # Get gradient w.r.t. interpolates
        gradients = autograd.grad(
            outputs=d_interpolates,
            inputs=interpolates,
            grad_outputs=fake,
            create_graph=True,
            retain_graph=True,
            only_inputs=True,
        )[0]
        gradients = gradients.view(gradients.size(0), -1)
        gradient_penalty = ((gradients.norm(2, dim=1) - 1) ** 2).mean()

        return gradient_penalty

In [ ]:
encode('tp/data/train.csv',batch_size=10000,total_size=50000)
#gc.collect()

Initializing model in test mode.
Loading model.
Loading finished in 15 seconds.
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Encoder_Inputs (InputLayer)     [(None, 72, 29)]     0                                            
__________________________________________________________________________________________________
mol_to_latent_model (Model)     (None, 512)          3226112     Encoder_Inputs[0][0]             
__________________________________________________________________________________________________
Decoder_Inputs (InputLayer)     [(None, 71, 29)]     0                                            
__________________________________________________________________________________________________
latent_to_states_model (Model)  [(None, 512), (None, 2117632     mol_to_latent_model[1][0]        
______________

In [ ]:
#decode('latent_rep')

In [ ]:
trainer = TrainModelRunner(
    input_data_path='latent_rep',n_epochs=1,
    decode_mols_save_path='decoded_smiles.smi')

In [ ]:
trainer.run()

Training of GAN started.


  7%|▋         | 69/937 [00:00<00:34, 25.23it/s]

In [ ]:
!nvcc --version

In [ ]:
decode('output/sampled.json',input_size=50000,output_smiles_file_path='bruuuh')

In [ ]:
with open('output/sampled.json', "r") as file:
  teste_load = json.loads(file.readline())

In [ ]:
teste_load